In [5]:
from google.cloud import bigquery, storage
import fastavro

In [ ]:
bq_project = "evident-data-dev"

backup_project = "evident-data-dev-backups"
backup_bucket = "lucas-test-exports"

full_table_name = "N/A"

try:

    bq_client = bigquery.Client(project=bq_project)
    datasets = bq_client.list_datasets()

    job_config = bigquery.ExtractJobConfig()
    job_config.destination_format = bigquery.DestinationFormat.AVRO
    job_config.use_avro_logical_types = True

    for dataset in datasets: 

        tables = bq_client.list_tables(dataset.dataset_id)
        for table in tables:

            full_table_name = f"{dataset.dataset_id}.{table.table_id}"

            if table.table_type != "TABLE":
                print(f"Skipping {full_table_name} as it is a {table.table_type}, not a standard TABLE.")
                continue

            gcs_uri = f"gs://{backup_project}/{backup_bucket}/{dataset.dataset_id}/{table.table_id}-*.avro"
            
            extract_job = bq_client.extract_table(
            table,
            gcs_uri,
            job_config=job_config)

            if extract_job.state == "DONE":
                if extract_job.errors:
                    error_msg = f"Extract job completed with errors: {extract_job.errors}"
                    print(error_msg)
                else:
                    print(f"Successfully backed up table {full_table_name} to {gcs_uri[5:-7]}")

            else:
                error_msg = f"Extract job in unexpected state: {extract_job.state}"
                print(error_msg)
    print("/// Back up job completed successfully ///")
   

except Exception as e:
        error_msg = f"Error backing up table {full_table_name}: {str(e)}"
        print(error_msg)




Successfully backed up table curated_business_metrics.monthly_okr_tracking to evident-data-dev-backups/lucas-test-exports/curated_business_metrics/monthly_okr_tracking
Successfully backed up table curated_google_analytics.member_hub_events to evident-data-dev-backups/lucas-test-exports/curated_google_analytics/member_hub_events
Successfully backed up table curated_google_patents.banking_patents to evident-data-dev-backups/lucas-test-exports/curated_google_patents/banking_patents
Successfully backed up table curated_google_patents.insurance_patents to evident-data-dev-backups/lucas-test-exports/curated_google_patents/insurance_patents
Successfully backed up table curated_google_patents.patents to evident-data-dev-backups/lucas-test-exports/curated_google_patents/patents
Successfully backed up table curated_google_patents.payments_patents to evident-data-dev-backups/lucas-test-exports/curated_google_patents/payments_patents
Skipping curated_google_scholar.ai_keyword_list as it is a EXTER